In [6]:
from PIL import Image
from io import BytesIO
import numpy as np

# Your raw bytes data
tiff_bytes = b'MM\x00*\x00\x00\x00\x08\x00\x11\x01\x00\x00\x03\x00\x00\x00\x01\x00\t\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\x00\t\x00\x00\x01\x02\x00\x03\x00\x00\x00\x03\x00\x00\x00\xdc\x01\x03\x00\x03\x00\x00\x00\x01\x80\xb2\x00\x00\x01\x06\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x01\x15\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\xe4\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\xec\x01(\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01B\x00\x03\x00\x00\x00\x01\x01\x00\x00\x00\x01C\x00\x03\x00\x00\x00\x01\x01\x00\x00\x00\x01D\x00\x04\x00\x00\x00\x01\x00\x00\x01\x9c\x01E\x00\x04\x00\x00\x00\x01\x00\x00\x06\x0b\x01S\x00\x03\x00\x00\x00\x03\x00\x00\x00\xf4\x85\xd8\x00\x0c\x00\x00\x00\x10\x00\x00\x00\xfc\x87\xaf\x00\x03\x00\x00\x00\x10\x00\x00\x01|\xa4\x81\x00\x02\x00\x00\x00\x040.0\x00\x00\x00\x00\x00\x00\x00\x00@\x00@\x00@\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x03\x00\x03\x00\x03\x00\x00?\x17\x8c}\xe8\x1e\x8c&\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@R\xf9\xd4\x91FU\x12\x00\x00\x00\x00\x00\x00\x00\x00\xbf\x17\x8c}\xe8\x1e\x8c&\x00\x00\x00\x00\x00\x00\x00\x00@3\x92\xb8#w)\xe5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\xf0\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\x02\x00\x03\x04\x00\x00\x00\x00\x01\x00\x02\x04\x01\x00\x00\x00\x01\x00\x01\x08\x00\x00\x00\x00\x01\x10\xe6x\xda\xed\xc11\x01\x00\x00\x00\xc2\xa0\xf5Om\n?\xa0' + b'\x00' * 1000  # Fill with rest of zeros


# Method 1: Direct PIL with explicit format
img_buffer = BytesIO(tiff_bytes)
with Image.open(img_buffer, formats=['TIFF']) as img:
    img.load()  # Force load the compressed data
    print(f"Image opened: {img.size}, mode: {img.mode}, format: {img.format}")
    img.show()
    # Convert to array
    img_array = np.array(img)
    print(f"Array shape: {img_array.shape}, dtype: {img_array.dtype}")
    print(f"Array values: min={img_array.min()}, max={img_array.max()}")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x109caa020>

In [6]:
import requests
import tifffile

def read_tiff_from_url(url):
    """
    Reads a TIFF image from a URL and returns it as a NumPy array.

    Args:
        url: The URL of the TIFF image.

    Returns:
        A NumPy array representing the TIFF image data, or None if an error occurred.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        with open("temp.tiff", "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        image = tifffile.imread("temp.tiff")
        return image
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None
    except Exception as e:
        print(f"Error reading image: {e}")
        return None
    finally:
        # Clean up the temporary file
        try:
            import os
            os.remove("temp.tiff")
        except FileNotFoundError:
            pass

# Example usage:
image_url = "https://earthengine.googleapis.com/v1/projects/amiable-raceway-342517/thumbnails/11e1e705611a6d4855409851dbf676d6-95a8a662b70b2b251bdc786ee04d53a4:getPixels"  # Replace with the actual URL
image_data = read_tiff_from_url(image_url)

if image_data is not None:
    print("Image data shape:", image_data.shape)
    # You can now work with the image_data NumPy array
else:
    print("Failed to read the TIFF image.")

Image data shape: (9, 9, 3)


In [15]:
from PIL import Image
import numpy as np
img = Image.fromarray((image_data * 255).astype(np.uint8))

In [17]:
img.show()